In [ ]:
#Data Preparation
#This notebook demonstrates data loading, preprocessing, and exploratory analysis on legal documents.



from pathlib import Path
import pandas as pd
import re

DD = Path(r"/mnt/data/legal_corpus")

# Load text files
records = []
for fp in DD.rglob("*.txt"):
    text = fp.read_text(encoding="utf-8", errors="ignore")
    records.append({
        "doc_id": fp.stem,
        "filepath": str(fp),
        "text_raw": text
    })
df = pd.DataFrame(records)
print("Documents loaded:", len(df))
df.head()




In [ ]:

# Preprocessing of Data
def preprocess_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r"\n\s*\d+\s*\n","\n", text)  # removing page number
    text = re.sub(r"\s+"," ", text)
    return text.strip()

df["text_clean"] = df["text_raw"].apply(preprocess_text)
df.head()


In [ ]:
# Statistics
df["tokens"] = df["text_clean"].apply(lambda x: len(x.split()))
df["length"] = df["text_clean"].str.len()
df[["tokens", "length"]].describe()

In [ ]:
# Structural markers
df["has_section"] = df["text_clean"].str.contains(r"(section|clause|article|§)", case=False, na=False)
print("Docs with structural markers (%):", df["has_section"].mean() * 100)

In [ ]:
# Visualization of Data
import matplotlib.pyplot as plt

plt.hist(df["tokens"], bins=20)
plt.xlabel("Tokens per document")
plt.ylabel("Frequency")
plt.title("Token Distribution")
plt.show()


In [ ]:
#Docs Length Analysis

# Already have tokens and length columns
print("Average tokens:", df["tokens"].mean())
print("Max tokens:", df["tokens"].max())
print("Min tokens:", df["tokens"].min())

import matplotlib.pyplot as plt
plt.hist(df["tokens"], bins=20)
plt.title("Document Token Length Distribution")
plt.xlabel("Tokens")
plt.ylabel("Count")
plt.show()

In [ ]:
#Word Frequency Analysis

from collections import Counter
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words("english"))

def tokenize(text):
    return [w for w in re.findall(r"\b\w+\b", text) if w not in stop_words]

all_tokens = []
df["tokens_list"] = df["text_clean"].apply(tokenize)
for tokens in df["tokens_list"]:
    all_tokens.extend(tokens)

word_freq = Counter(all_tokens)
print("Top 20 words:", word_freq.most_common(20))

# Plot
top_words = word_freq.most_common(20)
words, freqs = zip(*top_words)
plt.bar(words, freqs)
plt.xticks(rotation=45)
plt.title("Top 20 Most Frequent Words")
plt.show()


In [ ]:
#Document Similarity with TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer(stop_words="english")
X = vectorizer.fit_transform(df["text_clean"])

# Cosine similarity matrix
similarity_matrix = cosine_similarity(X)
print("Similarity matrix shape:", similarity_matrix.shape)

# Example: Top similar docs for first document
import numpy as np
doc_id = 0
similarities = similarity_matrix[doc_id]
top_idx = np.argsort(similarities)[::-1][1:4]  # skip itself
print("Most similar docs to", df.loc[doc_id, "doc_id"], ":", df.loc[top_idx, "doc_id"].tolist())
+586